In [1]:
%autosave 0

Autosave disabled


In [2]:
import tensorflow as tf
from tensorflow import keras

In [12]:
# !wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing_model.h5

# !wget https://bit.ly/mlbookcamp-pants -O pant.jpg

In [9]:
model = keras.models.load_model("clothing_model.h5")

2022-12-19 14:37:27.467964: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-19 14:37:27.471705: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 14:37:27.484668: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [18]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [17]:
img = load_img("pant.jpg", target_size = (299,299))

In [19]:
x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [20]:
preds = model.predict(X)

2022-12-19 14:48:28.471687: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-19 14:48:28.503610: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1799995000 Hz


In [21]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [23]:
dict(zip(classes,preds[0]))

{'dress': -1.8798621,
 'hat': -4.7563095,
 'longsleeve': -2.359534,
 'outwear': -1.0892645,
 'pants': 9.903782,
 'shirt': -2.8261788,
 'shoes': -3.648309,
 'shorts': 3.2411528,
 'skirt': -2.612095,
 't-shirt': -4.852034}

# Convert Keras to TF-Lite

In [25]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('clothing_model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmprhz85xy5/assets


INFO:tensorflow:Assets written to: /tmp/tmprhz85xy5/assets
2022-12-19 14:51:54.797022: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-12-19 14:51:54.797407: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-19 14:51:54.812808: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.003ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-12-19 14:51:56.803763: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2022-12-19 14:51:56.803846: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.


In [28]:
import tensorflow.lite as tflite

In [30]:
interpreter = tflite.Interpreter(model_path = 'clothing_model.tflite')
interpreter.allocate_tensors()

In [34]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [36]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

In [38]:
tf_preds = interpreter.get_tensor(output_index)

In [39]:
dict(zip(classes,tf_preds[0]))

{'dress': -1.8798645,
 'hat': -4.756309,
 'longsleeve': -2.3595347,
 'outwear': -1.0892642,
 'pants': 9.903783,
 'shirt': -2.8261802,
 'shoes': -3.6483088,
 'shorts': 3.241155,
 'skirt': -2.6120963,
 't-shirt': -4.8520355}

# Removing TF dependancy

In [60]:
#!python3 -m pip install tflite-runtime

In [80]:
from PIL import Image
import tflite_runtime.interpreter as tflite

def preprocess_manual(x):
    x /= 127.5
    x -= 1.
    
    return x

In [81]:
import requests
from io import BytesIO

response = requests.get(url)
with Image.open(BytesIO(response.content)) as img:
    img = img.resize((299,299), Image.NEAREST)

/tmp/ipykernel_299/2370253096.py:6: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize((299,299), Image.NEAREST)


In [82]:
url = 'https://bit.ly/mlbookcamp-pants'

In [83]:
x = np.array(img, dtype = 'float32')
X = np.array([x])

X = preprocess_manual(X)

In [84]:
interpreter = tflite.Interpreter(model_path = 'clothing_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, X)
interpreter.invoke()

In [85]:
tf_preds = interpreter.get_tensor(output_index)

dict(zip(classes,tf_preds[0]))

{'dress': -1.8798653,
 'hat': -4.7563086,
 'longsleeve': -2.3595326,
 'outwear': -1.0892627,
 'pants': 9.903783,
 'shirt': -2.8261797,
 'shoes': -3.648309,
 'shorts': 3.2411575,
 'skirt': -2.6120954,
 't-shirt': -4.8520327}